In [1]:
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from openai import OpenAI

# Setting up my OpenAI client
OPENAI_API_KEY = "Use you API Key"
client = OpenAI(api_key=OPENAI_API_KEY)

# Loading the notification messages from file
def load_notifications(filepath="notifications.txt"):
    with open(filepath, "r", encoding="utf-8") as f:
        text = f.read()
    return [chunk.strip() for chunk in text.strip().split("\n\n") if chunk.strip()]

notification_chunks = load_notifications("notifications.txt")
print(f"Loaded {len(notification_chunks)} notification messages.")

# Creating embeddings and building a FAISS index for notifications
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
notification_embeddings = embedding_model.encode(notification_chunks, convert_to_numpy=True)

dimension = notification_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(notification_embeddings.astype("float32"))

with open("notification_chunks.json", "w", encoding="utf-8") as f:
    json.dump(notification_chunks, f, indent=2)
faiss.write_index(index, "notification_index.faiss")

print("Saved notification index.")

# Reading the session summary and partial transcript
with open("Chat Gpt S1 Bottom Up Summary.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_summary = f.read().strip()

with open("Session One.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_transcript = f.read().strip()

# Limiting the transcript to 700 words for processing
session_transcript = ' '.join(session_transcript.split()[:700])

# Retrieving top-k relevant candidate messages based on session summary
query_embedding = embedding_model.encode([session_summary], convert_to_numpy=True)
D, I = index.search(np.array(query_embedding).astype("float32"), k=3)
top_messages = [notification_chunks[i] for i in I[0]]

# Building system and user prompts for message selection
system_prompt = "You are a psychologist selecting a helpful message to send to a client between sessions. Use ACT (Acceptance and Commitment Therapy) principles."

user_prompt = f"""
Session Summary:
---
{session_summary}

Transcript (partial):
---
{session_transcript}

Candidate Messages:
---
{chr(10).join(top_messages)}

Task:
Choose the best message for this client or rewrite a better one.
It should be:
- Short and supportive
- Actionable (something the client can reflect on or practice)
- Grounded in ACT processes (defusion, acceptance, values, committed action)
"""

# Asking the model for the best between-session message
response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.4,
    max_tokens=300,
)

# Printing the suggested message
suggested_message = response.choices[0].message.content.strip()
print("\nSuggested Between-Session Message:\n")
print(suggested_message)


Loaded 10 notification messages.


C:\Users\Gautham Rajesh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Saved notification index.

 Suggested Between-Session Message:

**Message:**

"Remember, pain is a normal part of life. Instead of avoiding it, try to make space for it. This week, notice when uncomfortable emotions arise and practice accepting them without judgment. Reflect on how you can respond in a way that aligns with your values, even in the presence of discomfort. You're building a meaningful life, one step at a time."


In [5]:
from openai import OpenAI
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

# Setting up Hugging Face API and DeepSeek model
HF_TOKEN = "Use you API Key"
MODEL_ID = "deepseek-ai/DeepSeek-R1-0528:novita"

# Initializing Hugging Face client with OpenAI compatibility
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
)

# Loading all notification messages from file
def load_notifications(filepath="notifications.txt"):
    with open(filepath, "r", encoding="utf-8") as f:
        text = f.read()
    return [chunk.strip() for chunk in text.strip().split("\n\n") if chunk.strip()]

notification_chunks = load_notifications("notifications.txt")
print(f"Loaded {len(notification_chunks)} notification messages.")

# Creating embeddings and FAISS index for the notifications
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
notification_embeddings = embedding_model.encode(notification_chunks, convert_to_numpy=True)

dimension = notification_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(notification_embeddings.astype("float32"))

# Saving the index and notification metadata for later use
with open("notification_chunks.json", "w", encoding="utf-8") as f:
    json.dump(notification_chunks, f, indent=2)
faiss.write_index(index, "notification_index.faiss")

print("Saved notification index.")

# Loading the session summary and transcript
with open("Deepseek S1 Bottom Up Summary.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_summary = f.read().strip()

with open("Session One.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_transcript = f.read().strip()

# Limiting transcript length to about 700 words for processing
session_transcript = ' '.join(session_transcript.split()[:700])

# Retrieving top-k relevant notification messages using FAISS
query_embedding = embedding_model.encode([session_summary], convert_to_numpy=True)
D, I = index.search(np.array(query_embedding).astype("float32"), k=3)
top_messages = [notification_chunks[i] for i in I[0]]

# Creating system and user prompts with session and candidate messages
system_prompt = "You are a psychologist selecting a helpful message to send to a client between sessions. Use ACT principles."

user_prompt = f"""
Session Summary:
---
{session_summary}

Transcript (partial):
---
{session_transcript}

Candidate Messages:
---
{chr(10).join(top_messages)}

Task:
Choose the best message for this client or rewrite a better one. It should be short, actionable, and grounded in ACT.
"""

# Sending the request to the DeepSeek model for message suggestion
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.4,
    max_tokens=300,
)

# Displaying the suggested between-session message
suggested_message = response.choices[0].message.content.strip()
print("\nSuggested Between-Session Message:\n")
print(suggested_message)


Loaded 10 notification messages.
Saved notification index.

 Suggested Between-Session Message:

<think>
Alright, let me process this request carefully. The client is in a delicate place - they're intellectually skeptical about the therapy process but willing to engage tentatively. The core tension is between their need for logical understanding ("if you can't describe it, you don't understand it") and the therapist's invitation to embrace experiential learning ("progress comes first—then understanding").

The three candidate messages each target different ACT processes:
1. Self-as-context focuses on the observing self
2. Present-moment targets mindfulness
3. Values connects to motivation

Considering the session dynamics, the client's stated discomfort stems from cognitive fusion (over-analyzing the method) and experiential avoidance (disliking the "worried" feeling). A values-based message would be most potent here because:
- It validates their courage in proceeding despite discomfor

In [7]:
from openai import OpenAI
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

# Setting up Hugging Face API and Meta Llama model
HF_TOKEN = "Use you API Key"
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct:cerebras"

# Initializing Hugging Face client with OpenAI-compatible API
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
)

# Loading all notification messages from file
def load_notifications(filepath="notifications.txt"):
    with open(filepath, "r", encoding="utf-8") as f:
        text = f.read()
    return [chunk.strip() for chunk in text.strip().split("\n\n") if chunk.strip()]

notification_chunks = load_notifications("notifications.txt")
print(f"Loaded {len(notification_chunks)} notification messages.")

# Creating embeddings and FAISS index for the notification messages
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
notification_embeddings = embedding_model.encode(notification_chunks, convert_to_numpy=True)

dimension = notification_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(notification_embeddings.astype("float32"))

# Saving index and notification chunks for later use
with open("notification_chunks.json", "w", encoding="utf-8") as f:
    json.dump(notification_chunks, f, indent=2)
faiss.write_index(index, "notification_index.faiss")

print("Saved notification index.")

# Loading session summary and transcript files
with open("Meta S1 Bottom Up Summary.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_summary = f.read().strip()

with open("Session One.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_transcript = f.read().strip()

# Limiting transcript length to about 700 words for processing
session_transcript = ' '.join(session_transcript.split()[:700])

# Retrieving top-k relevant notification messages using FAISS search
query_embedding = embedding_model.encode([session_summary], convert_to_numpy=True)
D, I = index.search(np.array(query_embedding).astype("float32"), k=3)
top_messages = [notification_chunks[i] for i in I[0]]

# Constructing system and user prompts for the model
system_prompt = "You are a psychologist selecting a helpful message to send to a client between sessions. Use ACT principles."

user_prompt = f"""
Session Summary:
---
{session_summary}

Transcript (partial):
---
{session_transcript}

Candidate Messages:
---
{chr(10).join(top_messages)}

Task:
Choose the best message for this client or rewrite a better one. It should be short, actionable, and grounded in ACT.
"""

# Sending the request to the Meta Llama model for message suggestion
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.4,
    max_tokens=300,
)

# Printing the suggested between-session message
suggested_message = response.choices[0].message.content.strip()
print("\nSuggested Between-Session Message:\n")
print(suggested_message)


Loaded 10 notification messages.
Saved notification index.

 Suggested Between-Session Message:

Based on the session summary and the client's concerns, I would select the message:

"There's a part of you that observes experiences without judgment—that self is always available, no matter what thoughts arise."

This message is short, actionable, and grounded in ACT principles. It acknowledges the client's tendency to overthink and analyze, and gently reminds them that they have a part of themselves that can observe their experiences without judgment. This message is particularly relevant because the client has expressed a willingness to trust the therapist and take a leap of faith, and this message can help them cultivate a sense of self-awareness and acceptance.

This message is also consistent with the therapist's approach, which emphasizes the importance of being in the present moment and taking action towards goals, rather than getting bogged down in theory and analysis. By remindin

In [9]:
from openai import OpenAI
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

# Setting up Hugging Face API and Moonshot model
HF_TOKEN = "Use you API Key"
MODEL_ID = "moonshotai/Kimi-K2-Instruct:novita"

# Initializing Hugging Face client with OpenAI-compatible API
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
)

# Loading all notification messages from file
def load_notifications(filepath="notifications.txt"):
    with open(filepath, "r", encoding="utf-8") as f:
        text = f.read()
    return [chunk.strip() for chunk in text.strip().split("\n\n") if chunk.strip()]

notification_chunks = load_notifications("notifications.txt")
print(f"Loaded {len(notification_chunks)} notification messages.")

# Creating embeddings and FAISS index for the notification messages
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
notification_embeddings = embedding_model.encode(notification_chunks, convert_to_numpy=True)

dimension = notification_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(notification_embeddings.astype("float32"))

# Saving index and notification chunks for later use
with open("notification_chunks.json", "w", encoding="utf-8") as f:
    json.dump(notification_chunks, f, indent=2)
faiss.write_index(index, "notification_index.faiss")

print("Saved notification index.")

# Loading session summary and transcript files
with open("Moonshot S1 Bottom Up Summary.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_summary = f.read().strip()

with open("Session One.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_transcript = f.read().strip()

# Limiting transcript length to about 700 words for processing
session_transcript = ' '.join(session_transcript.split()[:700])

# Retrieving top-k relevant notification messages using FAISS search
query_embedding = embedding_model.encode([session_summary], convert_to_numpy=True)
D, I = index.search(np.array(query_embedding).astype("float32"), k=3)
top_messages = [notification_chunks[i] for i in I[0]]

# Constructing system and user prompts for the model
system_prompt = "You are a psychologist selecting a helpful message to send to a client between sessions. Use ACT principles."

user_prompt = f"""
Session Summary:
---
{session_summary}

Transcript (partial):
---
{session_transcript}

Candidate Messages:
---
{chr(10).join(top_messages)}

Task:
Choose the best message for this client or rewrite a better one. It should be short, actionable, and grounded in ACT.
"""

# Sending the request to the Moonshot model for message suggestion
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.4,
    max_tokens=300,
)

# Printing the suggested between-session message
suggested_message = response.choices[0].message.content.strip()
print("\nSuggested Between-Session Message:\n")
print(suggested_message)


Loaded 10 notification messages.
Saved notification index.

 Suggested Between-Session Message:

“Between now and our next meeting, notice one moment when the mind demands a map before you’ll move. Instead of solving the uncertainty, practice taking one small step while the worry is still talking—like skiing before the instructions feel complete. Let me know what you discover.”
